# A Price Prediction Web App - ETL Notebook 2

The central steps in the ETL process for this project are:  
1. convert state plane coordinates, from the Dep. of City Planning, into standard Lat Long coordinates (Notebook 1)
2. aggregate ~ 60 Dep. of Finance sales data sets - spread across five boroughs and twelve years (This notebook)
3. Merge the Dep. of Finance and Dep. of City Planning Datasets (This notebook)

### Locations of the three sources: 
1. NYC Dep. of City Planning (has GPS coordinates for all NYC properties):  
   http://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page
2. NYC Dep. of Finance (tracks sales of all NYC properties):  
   https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page
3. NYTimes Real Estate Section (current listings on the market):      
   https://www.nytimes.com/section/realestate  
   The web scraper I built to pull listings off of the nytimes website: 
   https://github.com/MDHRDY/A_NY_Times_Real_Estate_Web_Scraper

# Aggregate  Annualized Sales Datasets from the Dep. of Finance

### Outline
1 [Load Libraries and Dep. of Finance dataset, and Merge datasets](#load)  
2 [Save dataframe w/ converted coordinates to file](#to_file)  
3 [Load Dep of City Planning dataset and Merge with Sales dataset](#merge)  
4 [Merge Datasets and Save to File](#to_file)

<a id='load'></a>
### Load Libraries and Dep. of Finance datasets, and Merge datasets

In [8]:
import numpy as np
import pandas as pd
options.display.max_columns = 100
options.display.max_rows = 100

Load one dataset to initialize columns for master dataset

In [9]:
path = "/Users/michaelhardy/mdh_python/nyc_project/Annualized_nyc_sales/"
df_init = pd.read_excel(path + '2003_Bronx.xls')
df_mast = DataFrame(columns=df_init.iloc[2])

# initialize colums 
mast_columns = df_init.iloc[2].tolist()

Open all downloaded files and merge into master dataframe

In [11]:
dates = map(str, range(2003,2016))
boroughs = ['_bronx.xls', '_brooklyn.xls', '_manhattan.xls', 
                            '_queens.xls', '_statenisland.xls']

total_length = 0
adjust = 0

for date in dates:
    for borough in boroughs:
        f = date + borough
        df_temp = read_excel(path + f)
        
        # since 2011, xls files have had extra row in header
        if date == '2011': adjust = 1
        df_temp.columns = df_temp.iloc[2 + adjust]
        df_temp = df_temp.ix[3:, :]
        df_temp.columns = mast_columns
        
        df_temp.columns = mast_columns
        df_mast = df_mast.append(df_temp)

        # reporting ...
        #print f, df_temp.shape[0]
        total_length += df_temp.shape[0]
        
print "total lines: ", total_length

total lines:  1254059


In [13]:
print "master dataframe size: ", df_mast.shape

master dataframe size:  (1254059, 21)


<a id='to_file'></a>
### Write results to file: 

In [15]:
df_mast.to_csv('all_sales.csv', index=False)

<a id='merge'></a>

### Load Dep of City Planning dataset and match columns

GPS Coordinates dataset (from Notebook1):

In [27]:
path2 = "/Users/michaelhardy/mdh_python/nyc_project/"
df_lat_long = read_csv(path2 + 'Post_DI/csvs/All_tax_classes_Lat_Long_w_89_variables.csv', low_memory=False)
df_lat_long.shape

(834182, 89)

Merged Sales dataset (from above):

In [26]:
sales = read_csv('all_sales.csv', low_memory=False)
print "Shape of Sales Dataframe: ", sales.shape

Shape of Sales Dataframe:  (1254059, 21)


Observe three columns (Block, Borough and Lot) to merge datasets on. 

In [29]:
print 'lat long dataframe format: ',df_lat_long.columns.tolist()[0:3]
print 'sales dataframe format: ',sales.columns.tolist()[0:6]

lat long dataframe format:  ['Borough', 'Block', 'Lot']
sales dataframe format:  ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'BLOCK', 'LOT']


Convert column names from df_lat_long to caps to match column format in sales dataframe

In [30]:
ll_columns = df_lat_long.columns.tolist()
ll_columns[71] = 'BOROUGH'
ll_columns[1] = 'BLOCK'
ll_columns[2] = 'LOT'
df_lat_long.columns = ll_columns
df_lat_long.LOT = df_lat_long.LOT.astype(str)
df_lat_long.BOROUGH = df_lat_long.BOROUGH.astype(str)
df_lat_long.BLOCK = df_lat_long.BLOCK.astype(str)

In [31]:
print "Check 'Borough' for null values: ", sales.BOROUGH.isnull().sum()

Check 'Borough' for null values:  0


<a id='to_file'></a>

### Merge Datasets and save to file

In [33]:
df_89 = sales.merge(df_lat_long,how='inner', left_on = ['BOROUGH','BLOCK', 'LOT'], right_on = ['BOROUGH','BLOCK', 'LOT'])
df_89.shape

(928790, 107)

In [34]:
df_89.to_csv('merge_sales_and_gps_datasets_107_features_11_8_16.csv', index=False)